In [ ]:
#From the Mushroom binary classification problem: https://www.kaggle.com/uciml/mushroom-classification
# Mushroom Class Classifier ANN

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd

In [ ]:
# Importing the dataset
dataset = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [ ]:
# Encoding categorical data
#encoding y
from sklearn.preprocessing import LabelEncoder
labelencoder_y_1 = LabelEncoder()
#reshape y
y = np.reshape(y,(-1,1)) #label encoder requires 2-dim array
y[:,0] = labelencoder_y_1.fit_transform(y[:, 0])

In [ ]:
#encoding X
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
orginalNumOfColsOfX = X.shape[1]
for i in range(X.shape[1]): 
    currNumOfColsOfX = X.shape[1]
    if i == 0: 
        ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough', sparse_threshold=0)
    else:
        indexOfColumnToEncode = currNumOfColsOfX - orginalNumOfColsOfX + i
        ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [indexOfColumnToEncode])], remainder='passthrough', sparse_threshold=0)
    X = np.array(ct.fit_transform(X)) 
    X = X[:, 1:] #get rid of dummy variable

In [ ]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#Feature Scaling i.e Standardisation or Normalisation
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#Making the ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Initialising the ANN
classifier = Sequential()
# Adding first hidden layer
classifier.add(Dense(units = 48, kernel_initializer = 'uniform', activation = 'relu'))
# Adding the second hidden layer
classifier.add(Dense(units = 48, kernel_initializer = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train.astype('float64'), batch_size = 10, epochs = 100) #ValueError if y_train datatype not changed

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
for i in range(y_pred.shape[0]):
    if y_pred[i,0] > 0.5: #using 50% as a threshold
        y_pred[i,0] = 1
    else:
        y_pred[i,0] = 0
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
y_test = y_test.astype('float32') #to match y_pred's datatype
cm = confusion_matrix(y_test, y_pred)

In [ ]:
#Evaluating our classification model
scores = classifier.evaluate(X_test, y_test)
print('Scores accuracy: %.2f%%' % (scores[1]*100))
#Using our confusion matrix
accuracy = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0])
print('Confusion matrix accuracy: %.2f%%' % (accuracy*100))
precision = cm[0,0]/(cm[0,0]+cm[0,1])
print('Confusion matrix precision: %.2f%%' % (precision*100))
recall = cm[0,0]/(cm[0,0]+cm[1,0])
print('Confusion matrix recall: %.2f%%' % (recall*100))
f1Score = 2*precision*recall/(precision+recall)
print('Confusion matrix f1Score: %.2f%%' % (f1Score*100))